## Ungraded Lab: Implementing ResNet

In this lab, you will continue exploring Model subclassing by building a more complex architecture.

Residual Networks make use of skip connections to make deep models easier to train.

There are branches as well as many repeating blocks of layers in this type of network.
You can define a model class to help organize this more complex code, and to make it easier to re-use your code when building the model.
As before, you will inherit from the nn.Module class so that you can make use of the other built-in methods that PyTorch provides.

## Imports

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cpu


## Implement Model subclasses

As shown in the lectures, you will first implement the Identity Block which contains the skip connections (i.e. the add() operation below. This will also inherit the Module class and implement the __init__() and forward() methods.

In [3]:
class IdentityBlock(nn.Module):
    def __init__(self, in_ch, out_ch, ksize=(3, 3)):
        super(IdentityBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=ksize, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        
        self.conv2 = nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=ksize, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        
        self.act = nn.ReLU()
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.act(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        x += inputs
        x = self.act(x)
        
        return x

From there, you can build the rest of the ResNet model.

You will call your IdentityBlock class two times below and that takes care of inserting those blocks of layers into this network.

In [4]:
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, padding=3)
        self.bn = nn.BatchNorm2d(64)
        self.act = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=(3,3))
        
        self.ind1a = IdentityBlock(in_ch=64, out_ch=64, ksize=3)
        self.ind1b = IdentityBlock(in_ch=64, out_ch=64, ksize=3)
        
        self.global_pool = nn.AvgPool2d(kernel_size=(8, 8))
        self.classifier = nn.Linear(in_features=64, out_features=num_classes)
        self.clf_act = nn.LogSoftmax(dim=1)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        # print(f"x1: {x.shape}")
        
        x = self.ind1a(x)
        # print(f"x_ind1a: {x.shape}")
        
        x = self.ind1b(x)
        # print(f"x_ind1b: {x.shape}")
        
        x = self.global_pool(x)
        # print(f"x_pool: {x.shape}")
        
        x = x.view(x.size(0), -1)
        # print(f"x_view: {x.shape}")
        
        x = self.classifier(x)
        # print(f"x_clf: {x.shape}")
        
        x = self.clf_act(x)
        # print(f"x_clf: {x.shape}")
        
        return x

In [5]:
x = torch.randn((32,1,28,28))
model = ResNet(num_classes=10)

In [6]:
pred = model(x)
pred.shape

torch.Size([32, 10])

## Dataset Preparation

In [7]:
# Image Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [8]:
# Load Dataset
train_data = MNIST(root='./', train=True, download=True, transform=transform)
test_data = MNIST(root='./', train=False, download=True, transform=transform)

In [9]:
# DataLoader
train_loader = DataLoader(dataset=train_data,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2,
                          pin_memory=True)
val_loader = DataLoader(dataset=test_data,
                        batch_size=32,
                        shuffle=True,
                        num_workers=2,
                        pin_memory=True)

## Training the Model

As mentioned before, inheriting the Model class allows you to make use of the other APIs that Keras provides, such as:
```
training
serialization
evaluation
```

You can instantiate a Resnet object and train it as usual like below:

In [10]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [11]:
# Train the Model
EPOCHS = 1

model.train()

for epoch in range(EPOCHS):
    running_loss = 0
    correct = 0
    
    for data in train_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
        running_loss += loss.item()
    
    print(f"Epoch: {epoch}, loss: {running_loss/len(train_loader)}, accuracy: {correct/len(train_loader.dataset)}")


# Evaluate Trained Model
running_loss = 0
correct = 0
    
model.eval()
for data in val_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    output = model(images)
    loss = criterion(output, labels)

    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

    running_loss += loss.item()

print(f"\nValidation - loss: {running_loss/len(val_loader)}, accuracy: {correct/len(val_loader.dataset)}")

Epoch: 0, loss: 0.1424209068339318, accuracy: 0.9629666805267334

Validation - loss: 0.058380220036287185, accuracy: 0.9837999939918518
